In [1]:
%matplotlib inline
import numpy as np
import sklearn
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt
import timeit
import pandas as pd
import math
import itertools
import time

import sklearn.neighbors
from sklearn.neighbors import (
    KernelDensity,
    KDTree,
)
from sklearn.preprocessing import (
    RobustScaler
)
import sklearn.decomposition
import sklearn.covariance

In [2]:
df = pd.read_csv(
    "../rawdata/mnist_train.csv",
    header=None,
).rename(
    columns={0: "label"}
)

In [13]:
np.random.seed(0)
nonzeroes = df[df.label > 0].values[:,1:785]
zeroes = df[df.label == 0].values[:,1:785]
np.random.shuffle(nonzeroes)
np.random.shuffle(zeroes)
sz = zeroes[:50]
snz = nonzeroes[:50]
print(len(nonzeroes))
print(len(zeroes))

combined = np.vstack([zeroes,snz])
print(len(combined))
real_outliers = np.concatenate([
        np.zeros(len(zeroes), dtype=bool),
        np.ones(len(snz), dtype=bool)
    ])

54077
5923
5973


In [4]:
def estimate_kde_bw(data):
    q3 = np.percentile(data, 75, axis=0)
    q1 = np.percentile(data, 25, axis=0)
    iqr = q3 - q1
    bw = iqr * (data.shape[0])**(-1.0/(data.shape[1]+4))
    return bw

In [5]:
def eval_scores(scores, true_outliers):
    cutoff = np.percentile(scores, 99)
    outliers = scores > cutoff
    both = sum(np.logical_and(outliers,true_outliers))
    predict = sum(outliers)
    actual = sum(true_outliers)
    print("{}/{},{}".format(both, predict, actual))
    return outliers

In [12]:
pca = sklearn.decomposition.PCA(
    n_components = 15,
    random_state = 0,
    svd_solver='randomized'
).fit(df.values[:,1:785])

In [14]:
rdata = pca.transform(combined)
rdata.shape

(5973, 15)

# Covariance

In [15]:
mcd = sklearn.covariance.EmpiricalCovariance(
)
mcd.fit(rdata)
mcd_scores = mcd.mahalanobis(rdata)
outliers = eval_scores(mcd_scores, real_outliers)

34/60,50


# KN

In [16]:
tree = KDTree(
    rdata
)

In [22]:
k=3
distances, _ = tree.query(rdata, k=k)
knndistances = distances[:,k-1]
outliers = eval_scores(knndistances, real_outliers)

35/60,50
